To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/deepseek-llm-7b-chat", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model.config.max_position_embeddings = 8192

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: deepseek-ai/deepseek-llm-7b-chat can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 8192!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

deepseek-ai/deepseek-llm-7b-chat does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `ChatML` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

Note ShareGPT uses `{"from": "human", "value" : "Hi"}` and not `{"role": "user", "content" : "Hi"}`, so we use `mapping` to map it.

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
from huggingface_hub import login

dataset = load_dataset("BlackAntik/dataset_ft_enhanced", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Let's see how the `ChatML` format works by printing the 5th element

In [ ]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': 'Ты --- эксперт по знакомствам, который берёт интервью с человеком и основываясь на нём, пишет для этого человека привлекательную анкету для сайта знакомств на русском языке от первого лица. Это должен быть сплошной текст без абзацов, от 15 до 75 слов.\n Интервью: - Кем ты работаешь/на кого учишься? - учился на повара кондитера\n- Чем ты занимаешься в свободное время? - занимаюсь спортом и стрельбой из лука, люблю читать, хобби нумизматика\n- Зачем ты здесь, что ищешь? - ищу Девушку для общения с перспективой отношений\n- Как бы ты описал себя тремя словами? - вкусно готовлю, занимаюсь спортом, люблю читать\n- Абсолютно любой интересный факт про тебя? - могу накормить\n'},
 {'from': 'gpt',
  'value': 'Учился на повара кондитера, вкусно готовлю, могу накормить. Занимаюсь спортом и стрельбой из лука, люблю читать, хобби нумизматика. Ищу девушку для общения с перспективой отношений.'}]

In [ ]:
dataset[5]["text"]

'<|im_start|>user\nТы --- эксперт по знакомствам, который берёт интервью с человеком и основываясь на нём, пишет для этого человека привлекательную анкету для сайта знакомств на русском языке от первого лица. Это должен быть сплошной текст без абзацов, от 15 до 75 слов.\n Интервью: - Кем ты работаешь/на кого учишься? - учился на повара кондитера\n- Чем ты занимаешься в свободное время? - занимаюсь спортом и стрельбой из лука, люблю читать, хобби нумизматика\n- Зачем ты здесь, что ищешь? - ищу Девушку для общения с перспективой отношений\n- Как бы ты описал себя тремя словами? - вкусно готовлю, занимаюсь спортом, люблю читать\n- Абсолютно любой интересный факт про тебя? - могу накормить\n<|im_end|>\n<|im_start|>assistant\nУчился на повара кондитера, вкусно готовлю, могу накормить. Занимаюсь спортом и стрельбой из лука, люблю читать, хобби нумизматика. Ищу девушку для общения с перспективой отношений.<|im_end|>\n'

If you're looking to make your own chat template, that also is possible! You must use the Jinja templating regime. We provide our own stripped down version of the `Unsloth template` which we find to be more efficient, and leverages ChatML, Zephyr and Alpaca styles.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

In [ ]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.98 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 37,478,400/7,000,000,000 (0.54% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.867800
2,1.928800
3,1.908600
4,1.988400
5,1.952400
6,1.800700
7,1.895700
8,1.935400
9,1.817700
10,1.932000


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

651.5022 seconds used for training.
10.86 minutes used for training.
Peak reserved memory = 9.98 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 67.702 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `ChatML`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Ты --- эксперт по знакомствам, который берёт интервью с человеком и основываясь на нём, пишет для этого человека привлекательную анкету для сайта знакомств на русском языке от первого лица. Это должен быть сплошной текст без абзацов, от 15 до 75 слов. Интервью: \n - Кем ты работаешь/на кого учишься? - Учусь на программиста\n - Чем ты занимаешься в свободное время? - Играю в баскетбол и смотрю советские фильмы\n - Зачем ты здесь? - Ищу мать моих будущих детей \n - Как бы ты описал себя тремя словами? - Целеустремлённый, умный, весёлый \n - Любой интересный факт про тебя? - Летал на параплане\n" },
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).int().to("cuda")


outputs = model.generate(input_ids = inputs, attention_mask=attention_mask, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

['<|im_start|>user\nТы --- эксперт по знакомствам, который берёт интервью с человеком и основываясь на нём, пишет для этого человека привлекательную анкету для сайта знакомств на русском языке от первого лица. Это должен быть сплошной текст без абзацов, от 15 до 75 слов. Интервью: \n - Кем ты работаешь/на кого учишься? - Учусь на программиста\n - Чем ты занимаешься в свободное время? - Играю в баскетбол и смотрю советские фильмы\n - Зачем ты здесь? - Ищу мать моих будущих детей \n - Как бы ты описал себя тремя словами? - Целеустремлённый, умный, весёлый \n - Любой интересный факт про тебя? - Летал на параплане\n<|im_end|>\n<|im_start|>assistant\nТвой интервью должно быть на русском языке и состоять из следующих вопросов:\n\n1. Кем ты работаешь/на кого учишься?\n2. Чем ты занимаешся в свободное время?\n3. Зачем ты здесь?\n4. Как бы ты описал себя тремя словами?\n5. Любой интересный факт про тебя?\n\nОтветы на эти вопросы должны быть написаны от первого лица, как будто этот человек говор

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Ты --- эксперт по знакомствам, который берёт интервью с человеком и основываясь на нём, пишет для этого человека привлекательную анкету от первого лица для сайта знакомств на русском языке от первого лица. Это должен быть сплошной текст без абзацов, от 15 до 75 слов. Интервью: \n - Кем ты работаешь/на кого учишься? - Учусь на программиста\n - Чем ты занимаешься в свободное время? - Играю в баскетбол и смотрю советские фильмы\n - Зачем ты здесь? - Ищу мать моих будущих детей \n - Как бы ты описал себя тремя словами? - Целеустремлённый, умный, весёлый \n - Любой интересный факт про тебя? - Летал на параплане\n" },
]
"""messages = [
    {"role": "user", "content": "Create a short story about elephants" },
]"""
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512, use_cache = True)

<|im_start|>user
Ты --- эксперт по знакомствам, который берёт интервью с человеком и основываясь на нём, пишет для этого человека привлекательную анкету от первого лица для сайта знакомств на русском языке от первого лица. Это должен быть сплошной текст без абзацов, от 15 до 75 слов. Интервью: 
 - Кем ты работаешь/на кого учишься? - Учусь на программиста
 - Чем ты занимаешься в свободное время? - Играю в баскетбол и смотрю советские фильмы
 - Зачем ты здесь? - Ищу мать моих будущих детей 
 - Как бы ты описал себя тремя словами? - Целеустремлённый, умный, весёлый 
 - Любой интересный факт про тебя? - Летал на параплане
<|im_end|>
<|im_start|>assistant
Текст:
Меня зовут Тимур и я студент. У меня много интересного, но особенно любимым занятием является баскетбол. В свободное время я смотрю хорошие советские фильмы. У меня очень конкретные планы - найти жену, с которой буду искать мои будущих детей.

Я - энергичный и целеустремлённый человек. Я обладую хорошим размышлением и эффективно реш

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|im_start|>user
What is a famous tall tower in Paris?<|im_end|> 
<|im_start|>assistant
The Eiffel Tower is a famous tall tower in Paris. It is one of the most recognizable landmarks in the world and is a popular tourist destination. The tower was built in 1889 for the World's Fair and is named after Gustave Eiffel, the engineer who designed and built it. The tower stands at a height of 324 meters (1,063 feet) and is made of iron. It is located on the Champ de Mars, a large public park in Paris.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoModelForPeftCausalLM
    from transformers import AutoTokenizer

    model = AutoModelForPeftCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")